# Data Mining – Assignment 1
---
> Chalkiopoulos Georgios, Electrical and Computer Engineer NTUA <br />
> Data Science postgraduate Student <br />
> gchalkiopoulos@aueb.gr

# Imports

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from pathlib import Path
from pprint import pprint
from typing import Dict
from random import randint

from helper_functions import compute_jaccard, primes, generate_hashing, compute_jaccard_hash

# 1) Import and pre-process the dataset with users

There are 3 files for the dataset,
* the users.txt file contains id, age, gender, occupation and postcode separated by |,
* the movies.txt file contains id, title (with release year) and some other information not related with the assignment separated by |,
* the ratings.txt file (tab separated) which contains userid, movieid, rating (1-5) and timestamp.

For this assignment only the set of movies that a user has rated, and not the ratings, will be used. In your report you should describe in detail any processing and conversion you made to the original data and the reasons it was necessary.

In [2]:
# Define phat
path = Path.cwd() / "MovieLensDataset"

In [3]:
# Users
users_col: list = ["id", "age", "gender", "occupation", "postcode"]
users_df: DataFrame = pd.read_csv(path / "users.txt", delimiter="|",names=users_col, header=None)
users_df.head()

,id,age,gender,occupation,postcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
# Movies df
movies_col: list = ["id", "title_year"]
movies_df: DataFrame = pd.read_csv(path / "movies.txt", delimiter="|", encoding='latin-1', header=None, names=movies_col, usecols=movies_col)
movies_df.head()

,id,title_year
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
# Ratings df
ratings_col: list = ["userid", "movieid", "rating", "timestamp"]
ratings_df: DataFrame = pd.read_csv(path / "ratings.txt", delimiter="\t", header=None, names=ratings_col)

print(f"Distinct users: {ratings_df['userid'].unique().shape[0]}")
print(f"Distinct Movies: {ratings_df['movieid'].unique().shape[0]}")
print("userid 1:")
ratings_df.loc[ratings_df.userid == 1]

Distinct users: 943
Distinct Movies: 1682
userid 1:


,userid,movieid,rating,timestamp
202,1,61,4,878542420
305,1,189,3,888732928
333,1,33,4,878542699
334,1,160,4,875072547
478,1,20,4,887431883
...,...,...,...,...
92049,1,28,4,875072173
92487,1,172,5,874965478
94019,1,122,3,875241498
96699,1,152,5,878542589


# 2) Compute exact Jaccard similarity of users

* As a first step we will create a set, for each user, containing the movies he has rated.

In [6]:
# Create list of movies per user
user_movies: DataFrame =  ratings_df[["userid", "movieid"]].groupby('userid').apply(lambda x: set([i for i in x['movieid']]))
user_movies.head()

userid
1    {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
2    {257, 258, 1, 10, 13, 14, 269, 272, 273, 274, ...
3    {258, 260, 264, 268, 271, 272, 288, 294, 299, ...
4    {258, 260, 264, 11, 271, 288, 294, 300, 301, 3...
5    {1, 2, 17, 21, 24, 25, 29, 40, 42, 50, 62, 63,...
dtype: object

Using this set, we will compute the jaccard scores between all possible pairs of users, and keep those whose similarity is above 50%.

In [31]:
# compute jaccard scores
jaccard_scores = compute_jaccard(data=user_movies)

In [48]:
print("Similar Pairs: ")
similar_users = []
for user1, sim_users in jaccard_scores.items():
    for user_score in sim_users:
        similar_users.append({user1, user_score[0]})
        print(f"Pair: ({user1}, {user_score[0]}) - Jaccard Score: {user_score[1]*100:.2f}%")
        jaccard_scores

Similar Pairs: 
Pair: (408, 898) - Jaccard Score: 83.87%
Pair: (328, 788) - Jaccard Score: 67.30%
Pair: (489, 587) - Jaccard Score: 62.99%
Pair: (600, 826) - Jaccard Score: 54.55%
Pair: (451, 489) - Jaccard Score: 53.33%
Pair: (674, 879) - Jaccard Score: 52.17%
Pair: (554, 764) - Jaccard Score: 51.70%
Pair: (197, 826) - Jaccard Score: 51.30%
Pair: (197, 600) - Jaccard Score: 50.00%
Pair: (800, 879) - Jaccard Score: 50.00%


### Output the movie titles that the most similar pair of users has seen.
The most similar pair of users are 408 and 898 with a Jaccard score of 83.87. The movies that these users have seen is presented below:

In [51]:
# join the ratings df with the movies to get the movie names per user id
joined: DataFrame = ratings_df.loc[ratings_df.userid.isin([408, 898])].merge(movies_df, left_on="movieid", right_on="id")

print("Movies seen by users 408 and/or 898: ")
for movie in joined["title_year"].unique():
    print(joined.loc[joined.title_year == movie].userid.values, end=" ")
    print(movie)

Movies seen by users 408 and/or 898: 
[408 898] Jackal, The (1997)
[408 898] Rocket Man (1997)
[408 898] Saint, The (1997)
[408 898] Contact (1997)
[408 898] Air Force One (1997)
[408 898] Everyone Says I Love You (1996)
[408 898] English Patient, The (1996)
[408 898] Scream (1996)
[408 898] Apt Pupil (1998)
[408 898] Wag the Dog (1997)
[408 898] Mouse Hunt (1997)
[408 898] Spawn (1997)
[408 898] Midnight in the Garden of Good and Evil (1997)
[408 898] L.A. Confidential (1997)
[408 898] Titanic (1997)
[408 898] Lost Highway (1997)
[408 898] Gattaca (1997)
[408 898] Indian Summer (1996)
[408 898] Conspiracy Theory (1997)
[408] Liar Liar (1997)
[408 898] Starship Troopers (1997)
[408 898] Cop Land (1997)
[898 408] Tomorrow Never Dies (1997)
[898 408] Kolya (1996)
[408 898] Good Will Hunting (1997)
[408 898] U Turn (1997)
[898] Jungle2Jungle (1997)
[898] Alien: Resurrection (1997)
[408 898] Rainmaker, The (1997)
[898] As Good As It Gets (1997)
[898] Deceiver (1997)


# 3) Compute similarity using Min-hash signatures

Description of hash functions: use the following family of hash functions: $h_{a,b}(x)=(ax+b) mod R$, with a,b random integers in the interval (0,R) and R a large enough prime number that you may want to finetune in your initial experimentation. Make sure that each hash function uses different values of a,b pairs.

Evaluation of Min-hashing: Use 50, 100, and 200 hash functions. For each value, output the pair of users that have estimated similarity at least 0.5, and report the number of false positives and false negatives (against the exact Jaccard similarity) that you obtain. For the false positives and negatives, report the averages for 5 different runs using different functions. Comment on how the number of hash functions affects the false positive and negatives figures.

### Create user matrices

Before proceeding, we will create the "Document" of each user. This, in practice, is a list in which we assign the number 1 if the user has seen the movie with id i and 0 if he hasn't.

* keep in mind that the movie id's start from one while python indices start from zero

In [10]:
# Create doc
movies = user_movies.apply(lambda x: [1 if i+1 in x else 0 for i in range(ratings_df.movieid.max())])
movies

userid
1      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2      [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
5      [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
939    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...
940    [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, ...
941    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...
942    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
943    [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...
Length: 943, dtype: object

* We need a large value of R which will make sure that we have at least 1682 different values (distinct movies). In this case we will avoid having two values for the same row (hash conflict), while hashing the rows movie ids.

In [11]:
# find the max value of the movieid
number_of_movies: int = ratings_df.movieid.max()
number_of_movies

1682

In [12]:
rows: np.ndarray = np.array(range(1, len(movies[1])+1))

for _prime in primes(100000):
    a, b = randint(0, _prime), randint(0, _prime)
    hashing = (a * rows + b) % _prime

    # break if number of unique rows is equal to the number of rows
    if len(np.unique(np.array(hashing))) >= number_of_movies:
        R: int = _prime
        break

print(f"Optimal value for R is: {_prime}")

Optimal value for R is: 1693


In [13]:
n = 50
hash_functions = generate_hashing(n=50, rows=rows, R=_prime, iteration=6)
signature = []

for i in range(n):
    hash_func = hash_functions[i]
    s = movies.apply(lambda x: np.where(np.array(x) == 1,  hash_func, 9999).min())
    signature.append(s)
signature = np.array(signature)

In [14]:
min_hash = pd.DataFrame(signature.tolist()).T
min_hash.index = np.arange(1, len(min_hash) + 1)
min_hash

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1,2,3,5,4,12,1,1,9,298,5,...,3,1,15,9,0,2,1,4,0,1
2,32,28,25,33,28,6,4,41,298,26,...,3,21,14,6,30,16,0,11,33,14
3,0,5,1,26,44,6,18,9,298,0,...,12,7,38,23,30,4,0,18,9,107
4,37,68,11,90,125,75,18,9,298,190,...,2,88,13,9,35,83,0,25,25,8
5,2,4,8,4,11,0,15,8,298,9,...,22,4,12,3,0,9,1,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,1,93,15,29,73,12,46,147,298,12,...,27,43,1,29,102,14,14,0,32,12
940,7,4,1,26,30,7,11,9,298,9,...,3,28,61,11,0,25,0,18,9,1
941,111,29,15,82,47,12,29,31,298,57,...,26,78,54,113,53,14,0,65,88,8
942,96,4,28,13,12,40,11,60,298,9,...,2,35,11,9,30,4,43,17,26,14


In [57]:
from itertools import combinations
from collections import defaultdict
import pandas as pd
from typing import Tuple, List
from random import shuffle, Random
import numpy as np

# create pairs (user1, user2)
pairs = list(combinations(list(min_hash.index), 2))
usim = defaultdict(dict)

doc_length = min_hash.shape[1]
for i, (user_1, user_2) in enumerate(pairs):

    s1: pd.Series = min_hash.loc[user_1]
    s2: pd.Series = min_hash.loc[user_2]

    common_movies = np.count_nonzero(s1 == s2)

    jacc: float = common_movies/doc_length

    if jacc >= 0.5 or {user_1, user_2} in similar_users:
        usim[user_1][user_2]: float = jacc

    if i % 50000 == 0 and i != 0:
        print(f"Evaluated {i}/{len(pairs)} pairs")

neighbors_u: dict = {user: sorted(usim[user].items(), key=lambda x: x[1], reverse=True) for user in usim}

scores: dict = dict(sorted(neighbors_u.items(), key=lambda item: item[1][0][1], reverse=True))

Evaluated 0 pairs
Evaluated 10000 pairs
Evaluated 20000 pairs
Evaluated 30000 pairs
Evaluated 40000 pairs
Evaluated 50000 pairs
Evaluated 60000 pairs
Evaluated 70000 pairs
Evaluated 80000 pairs
Evaluated 90000 pairs
Evaluated 100000 pairs
Evaluated 110000 pairs
Evaluated 120000 pairs
Evaluated 130000 pairs
Evaluated 140000 pairs
Evaluated 150000 pairs
Evaluated 160000 pairs
Evaluated 170000 pairs
Evaluated 180000 pairs
Evaluated 190000 pairs
Evaluated 200000 pairs
Evaluated 210000 pairs
Evaluated 220000 pairs
Evaluated 230000 pairs
Evaluated 240000 pairs
Evaluated 250000 pairs
Evaluated 260000 pairs
Evaluated 270000 pairs
Evaluated 280000 pairs
Evaluated 290000 pairs
Evaluated 300000 pairs
Evaluated 310000 pairs
Evaluated 320000 pairs
Evaluated 330000 pairs
Evaluated 340000 pairs
Evaluated 350000 pairs
Evaluated 360000 pairs
Evaluated 370000 pairs
Evaluated 380000 pairs
Evaluated 390000 pairs
Evaluated 400000 pairs
Evaluated 410000 pairs
Evaluated 420000 pairs
Evaluated 430000 pairs
Ev

In [15]:
# compute jaccard scores
jaccard_scores_hash: Dict[int, list] = compute_jaccard_hash(data=min_hash)
jaccard_scores_hash

IndexError: single positional indexer is out-of-bounds

In [63]:
for user_1, sim_users in scores.items():
    for (user_2, users_scores) in sim_users:
        if {user_1, user_2} in similar_users:
            if users_scores >= 0.5:
                print(f"True Positive: ({user_1}, {user_2})")
            if users_scores < 0.5:
                print(f"False Negative: ({user_1}, {user_2})")
        else:
            print(f"False Positive: ({user_1}, {user_2})")

True Positive: (408, 898)
True Positive: (197, 826)
True Positive: (197, 600)
False Positive: (258, 856)
False Positive: (258, 589)
False Positive: (100, 752)
True Positive: (600, 826)
False Positive: (111, 171)
False Positive: (111, 673)
False Positive: (111, 866)
True Positive: (328, 788)
True Positive: (489, 587)
False Positive: (356, 856)
False Positive: (474, 537)
False Positive: (474, 716)
False Positive: (13, 450)
False Positive: (37, 746)
False Positive: (37, 217)
False Positive: (70, 307)
False Positive: (70, 275)
False Positive: (70, 536)
False Positive: (284, 574)
False Positive: (313, 716)
False Positive: (359, 386)
False Positive: (379, 650)
False Positive: (429, 896)
True Positive: (554, 764)
False Positive: (18, 305)
False Positive: (217, 619)
False Positive: (406, 537)
False Positive: (406, 747)
False Positive: (673, 784)
True Positive: (674, 879)
True Positive: (800, 879)
False Positive: (804, 881)
False Positive: (85, 474)
False Positive: (120, 935)
False Positive: (1